Projet: Créer une application web VTC avec des utilisatrices et conductrices féminin

Thomas Papas
Python Software Engineer

Objectifs :

- Garentir une meillieure sécurité pour les femmes qui sortent le soir
- Créer une alternative face a l'expansion sauvage de l'uberisation
- 


Plan de la présentation

1. Configuration de la base de données avec Flask SQLAlchemy et de l'appli
2. Gestion des utilisateurs et présentation de l'appli
3. Présentation du projet
4. Questions 

1) Configuration de la base de données avec Flask SQLAlchemy et de l'appli :


- Création d'un fichr ".env" pour stocker nos clés secrètes. Permet notamment d'éviter de les push
- Configuration dans un fichier models 



   

Import des librairies python utilisées

In [3]:
import os
import sys 

#sys.path.append(os.path.dirname(os.path.dirname(os.path.realpath(__file__))))

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager,UserMixin
from werkzeug.security import generate_password_hash, check_password_hash
from flask_mail import Mail,Message
from dotenv import load_dotenv
from flask_jwt_extended import JWTManager
from _datetime import datetime 


Initialisation de SQLAlchemy, LoginManager, Flask Mail et Flask JWT extended (deeplinking)

In [6]:

db = SQLAlchemy()  # SQLAlchemy for database interactions
login_manager = LoginManager()  # Flask-Login for user session management
mail = Mail()  # Flask-Mail for email handling
jwt = JWTManager()  # Flask-JWT-Extended for JWT authentication


Data class de User

In [7]:
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(50), index=True, unique=True)
    email = db.Column(db.String(120), index=True)
    password_hash = db.Column(db.String(128))
    name = db.Column(db.String(50))
    surname = db.Column(db.String(50))
    gender = db.Column(db.String(50), default="female")
    role = db.Column(db.String(10))

    def __init__(self, username, name, surname, email, password,role):
        self.username = username
        self.name = name
        self.surname = surname
        self.email = email
        self.role = role
        self.set_password(password)

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
    
    def __repr__(self):
        return f'<User {self.id}>'

Project App : Application VTC avec des femmes conductrices et utilisatrices 

Plan de la présentation:

1) Configuration de la base de données avec Flask SQLAlchemy  et de l'appli
2) Gestion de l'invittion des utilisatrices et deeplinking
3) Présentation de l'appli
4) Questions

1) Configuration de la base de données avec Flask SQLAlchemy et de l'appli

- Utilisation d'un fichier .env pour les clés secretes (pour éviter qu'elles se fassent push)
- Trois Dataclass User, RideOrder, InvitationEmails

Import des librairies Python nécessaires pour la configuration

In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager,UserMixin
from werkzeug.security import generate_password_hash, check_password_hash
from flask_mail import Mail,Message
from dotenv import load_dotenv
from flask_jwt_extended import JWTManager
from _datetime import datetime 

Initialisation des extensions (fichier models.py)

In [ ]:
db = SQLAlchemy()  # SQLAlchemy for database interactions
login_manager = LoginManager()  # Flask-Login for user session management
mail = Mail()  # Flask-Mail for email handling
jwt = JWTManager()  # Flask-JWT-Extended for JWT authentication


Dataclass User

In [ ]:
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(50), index=True, unique=True)
    email = db.Column(db.String(120), index=True)
    password_hash = db.Column(db.String(128))
    name = db.Column(db.String(50))
    surname = db.Column(db.String(50))
    gender = db.Column(db.String(50), default="female")
    role = db.Column(db.String(10))

    def __init__(self, username, name, surname, email, password,role):
        self.username = username
        self.name = name
        self.surname = surname
        self.email = email
        self.role = role
        self.set_password(password)

    def set_password(self, password):
        self.password_hash = generate_password_hash(password)

    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
    
    def __repr__(self):
        return f'<User {self.id}>'


Dataclass RideOrder

In [ ]:
   
class RideOrder(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    departure = db.Column(db.String(200), nullable=False)
    destination = db.Column(db.String(200), nullable=False)
    time = db.Column(db.String(10), nullable=False,  default=lambda: datetime.utcnow().strftime('%H:%M:%S')) #Puts the current UTC time in the form HH:MM:SS as default
    status = db.Column(db.String(20), default='pending')  # Ride status
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=True)  # ID of the user who accepts the ride
    
    user = db.relationship('User', backref='ride_orders', lazy=True, foreign_keys=[user_id])

    



    def __repr__(self):
        return f'<RideOrder {self.id}>'


Dataclass InvitationEmails

In [ ]:
class InvitationEmails(db.Model):  
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(50))
    token = db.Column(db.String(128))

Initialisation de l'appli (fichier models.py)

In [ ]:
def create_app():
    # Load environment variables from .env file
    load_dotenv()

    # Initialize Flask application
    app = Flask(__name__)

    # Configure the Flask application with necessary settings
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////home/nomades/Documents/pse_2024_0405/flask/Project_App/instance/project_app.db'  # Database URI
    app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # Disable track modifications to save resources
    app.config['SECRET_KEY'] = os.getenv('SECRET_KEY')  # Secret key for session management and CSRF protection
    app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY')  # Secret key for JWT
    app.config['MAIL_SERVER'] = 'smtp.gmail.com'  # Mail server
    app.config['MAIL_PORT'] = 587  # Mail server port
    app.config['MAIL_USE_TLS'] = True  # Use TLS for secure email transmission
    app.config['MAIL_USE_SSL'] = False  # Do not use SSL (we are using TLS)
    app.config['MAIL_USERNAME'] = os.getenv('MAIL_USERNAME')  # Email username from environment variables
    app.config['MAIL_PASSWORD'] = os.getenv('MAIL_PASSWORD')  # Email password from environment variables
    
    

    # Initialize extensions with the Flask application
    login_manager.init_app(app)  # Initialize Flask-Login
    login_manager.login_view = 'users.user_login'  # Redirect not logged-in users to this page
    db.init_app(app)  # Initialize SQLAlchemy
    mail.init_app(app)  # Initialize Flask-Mail
    jwt.init_app(app)  # Initialize Flask-JWT-Extended

    # Create database tables if they don't exist
    with app.app_context():
        db.create_all()

    
   

    return app

app = create_app()

def generate_deeplink(token):
    # Generate the deeplink with the token
    deeplink = f"http://127.0.0.1:5000/register/{token}"
    return deeplink

def send_invitation_email(email, deeplink):
    subject = "Invitation à rejoindre notre service VTC"
    body = f"""
    Bonjour,
    
    Vous avez été invitée à rejoindre notre service VTC. Cliquez sur le lien ci-dessous pour créer votre compte :
    
    {deeplink}
    
    Cordialement,
    L'équipe VTC
    """
    
    msg = Message(subject, sender='thomaspapas470@gmail.com', recipients=[email])
    msg.body = body

    with app.app_context():
         mail.send(msg)
         

Import de app dans app.py ,des blueprints et des libraieries python utilisées 

In [2]:


from flask import  render_template,flash,redirect,url_for
from config.models import app,db, InvitationEmails,User
from flask_jwt_extended import decode_token
from forms import RegistrationForm

from admin_routes.admin_controller import admins  # Import admin blueprint
from user_routes.user_controller import users  # Import user blueprint
from driver_routes.driver_controller import drivers  # Import driver blueprint

app.register_blueprint(admins, url_prefix='/admins')  # Register admin routes under /admins
app.register_blueprint(users, url_prefix='/users')  # Register user routes under /users
app.register_blueprint(drivers, url_prefix='/drivers')  # Register driver routes under /drivers


@app.route("/")
def index():
    return render_template('base.html')

2) Gestion des Utilisateurs et deeplinking

In [ ]:
@app.route('/register/<token>', methods=['GET', 'POST'])
def register(token):

        try:

            # Decode the token to extract user information
            token_data = decode_token(token)

            # Extract the email and role from the decoded token data
            email = token_data['sub']['email']
            role = token_data['sub']['role']
        except Exception as e:
            # If an error occurs during token decoding (e.g., token is invalid or expired),
            # flash an error message to the user indicating that the token is invalid
            flash('Invalid token', 'danger')

            # Redirect the user to the home page (index) as the token is invalid
            return redirect(url_for('index'))

        # Fetch all invitation emails from the database
        db_emails = [inv.email for inv in InvitationEmails.query.all()]

        # Check if the email extracted from the token is in the list of invitation emails
        if email not in db_emails:
            # If the email is not in the invitation emails, show an error message
            flash('Invalid link', 'danger')
            
            # Redirect the user to the home page (index) as the link is invalid
            return redirect(url_for('index'))

        # Create a new instance of the registration form
        form = RegistrationForm()

        # Check if the form has been submitted and is valid
        if form.validate_on_submit():
            # Verify that the email provided in the form matches the email extracted from the token
            if email == form.email.data: 
                # Create a new user instance based on the role extracted from the token
                if role == 'admin':
                    user = User(username=form.username.data, email=form.email.data, password=form.password.data, name=form.name.data, surname=form.surname.data, role='admin')
                elif role == 'driver':
                    user = User(username=form.username.data, email=form.email.data, password=form.password.data, name=form.name.data, surname=form.surname.data, role='driver')
                else:
                    user = User(username=form.username.data, email=form.email.data, name=form.name.data, password=form.password.data, surname=form.surname.data, role='user')

                # Set the user's password
                user.set_password(form.password.data)

                # Add the new user to the database session
                db.session.add(user)

                # Commit the session to save the user to the database
                db.session.commit()

                # Flash a success message indicating that the user is now registered
                flash(f'You are now a registered {role}', 'success')

                # Redirect the user to the login page 
                return redirect(url_for('users.user_login'))

        # Render the registration template with the form and email context variables
        return render_template('register.html', form=form, email=email)
